# Exploratory: Check column null percentage

Summary: <br>
This notebook is used to explore the SDG&E weather station data that was extracted via Synopic API.

**Note: Reading FULL weather files are large and may exceed memory for computer. Computer used for execution contains 64GB memory with no issue. Add chunksize parameter as needed or analyze smaller station files.**

In [1]:
import json
from datetime import datetime, timedelta

import pandas as pd

pd.set_option("display.max_rows", None, "display.max_columns", None)

### Read in raw data

In [2]:
%%time
weather_sdge_df = pd.read_csv("../../data/processed/weather_SDGE.csv", header=[0, 1])
weather_hpwren_df = pd.read_csv(
    "../../data/processed/weather_HPWREN.csv", header=[0, 1]
)
weather_scedison_df = pd.read_csv(
    "../../data/processed/weather_SC-EDISON.csv", header=[0, 1]
)

CPU times: user 30.9 s, sys: 3.31 s, total: 34.2 s
Wall time: 34.2 s


### Helper functions to calculate attribute null + missing percentages and station null percentage

In [3]:
# Calculate total 10 minute intervals in timeframe

with open("config.json") as config_file:
    cfg = json.load(config_file)

# YYYYmmddHHMM (format needed for api)
START_DATETIME = cfg["weather_start_datetime"]
END_DATETIME = cfg["weather_end_datetime"]

start = datetime.strptime(START_DATETIME, "%Y%m%d%H%M")
end = datetime.strptime(END_DATETIME, "%Y%m%d%H%M")

elapsed = end - start
number_of_intervals = elapsed / timedelta(minutes=10)
number_of_intervals

268560.0

In [4]:
def calc_att_null_percentage(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return dataframe with null percentage of each column given a weather data dataframe.
    """
    rows = df.shape[0]
    return pd.DataFrame(df.isna().sum() / rows * 100, columns=["null_percentage"])

In [5]:
def calc_att_null_miss_percentage(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return dataframe with null or missing percentage of each column given a weather data dataframe.
    """
    # num stations x total number of intervals
    num_stations = len(df["Station_ID"]["Unnamed: 0_level_1"].unique())
    total_rows = num_stations * number_of_intervals

    # count non-NA cells for each column
    # null+missing/total
    # null+missing = total-actual
    return pd.DataFrame(
        (total_rows - df.count()) / total_rows * 100, columns=["null_miss_percentage"]
    )

In [6]:
def calc_station_null_percentage(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return dataframe with null percentage of each column for each station given a weather data dataframe.
    """
    tmp = df.set_index("Station_ID")

    # get null counts of each column for each station
    station_df = tmp.isna().groupby("Station_ID").sum()
    # update index
    # just take the station value in each list
    orig_index = station_df.index.to_list()
    new_index = [item[0] for item in orig_index]
    station_df.index = pd.Index(new_index, name="Station_ID")

    # get the total row count for each station
    counts_df = (
        tmp["Date_Time"]
        .groupby("Station_ID")
        .count()
        .rename(columns={"Unnamed: 1_level_1": "row_count"})
    )
    # update index
    # just take the station value in each list
    orig_index = counts_df.index.to_list()
    new_index = [item[0] for item in orig_index]
    counts_df.index = pd.Index(new_index, name="Station_ID")

    # calc null percentage
    station_null_per_df = station_df.copy()
    for station in counts_df.index:
        # print(station)
        # print(station_null_per_df.loc[station] / counts_df.loc[station]["row_count"])
        # print(counts_df.loc[station]["row_count"])
        station_null_per_df.loc[station] = (
            station_null_per_df.loc[station] / counts_df.loc[station]["row_count"] * 100
        )

    return station_null_per_df

### Calc null percentages for each network

#### SDGE

In [7]:
weather_sdge_df.shape

(24321594, 10)

In [8]:
%%time
calc_att_null_miss_percentage(weather_sdge_df).round(2)

CPU times: user 5.63 s, sys: 53.1 ms, total: 5.69 s
Wall time: 5.68 s


,,null_miss_percentage
Station_ID,Unnamed: 0_level_1,11.21
Date_Time,Unnamed: 1_level_1,11.21
air_temp_set_1,Celsius,11.36
relative_humidity_set_1,%,11.33
wind_speed_set_1,m/s,11.24
wind_gust_set_1,m/s,11.22
wind_direction_set_1,Degrees,14.61
dew_point_temperature_set_1d,Celsius,11.36
u,Unnamed: 8_level_1,14.63
v,Unnamed: 9_level_1,14.63


In [9]:
%%time
sdge_station_null_per_df = calc_station_null_percentage(weather_sdge_df)
sdge_station_null_per_df

CPU times: user 27.5 s, sys: 1.79 s, total: 29.2 s
Wall time: 29.2 s


,Date_Time,air_temp_set_1,relative_humidity_set_1,wind_speed_set_1,wind_gust_set_1,wind_direction_set_1,dew_point_temperature_set_1d,u,v
,Unnamed: 1_level_1,Celsius,%,m/s,m/s,Degrees,Celsius,Unnamed: 8_level_1,Unnamed: 9_level_1
Station_ID,,,,,,,,,
AMOSD,0,0.261685,0.227253,0.026781,0.000000,4.045022,0.488938,4.071803,4.071803
ARHSD,0,0.000000,0.001135,0.027992,0.000000,8.559983,0.001135,8.587975,8.587975
AVOSD,0,0.000000,0.000374,0.021668,0.000000,8.577844,0.000374,8.599511,8.599511
BFDSD,0,0.000000,0.000380,0.031921,0.000000,0.761163,0.000380,0.793084,0.793084
BLCSD,0,0.000000,0.000000,0.028577,0.000000,3.724844,0.000000,3.753422,3.753422
BLUSD,0,0.000000,0.001026,0.066680,0.000000,1.880386,0.001026,1.947066,1.947066
BMRSD,0,0.000000,0.000000,0.065547,0.000000,0.572511,0.000000,0.638058,0.638058
BMSSD,0,0.000376,0.001504,0.036859,0.000752,2.219456,0.001504,2.256315,2.256315


#### HPWREN

In [10]:
weather_hpwren_df.shape

(2456501, 10)

In [11]:
%%time
calc_att_null_miss_percentage(weather_hpwren_df).round(2)

CPU times: user 584 ms, sys: 13.2 ms, total: 597 ms
Wall time: 596 ms


,,null_miss_percentage
Station_ID,Unnamed: 0_level_1,42.83
Date_Time,Unnamed: 1_level_1,42.83
air_temp_set_1,Celsius,43.40
relative_humidity_set_1,%,43.40
wind_speed_set_1,m/s,43.87
wind_gust_set_1,m/s,43.87
wind_direction_set_1,Degrees,43.87
dew_point_temperature_set_1d,Celsius,43.40
u,Unnamed: 8_level_1,43.87
v,Unnamed: 9_level_1,43.87


In [12]:
%%time
hpwren_station_null_per_df = calc_station_null_percentage(weather_hpwren_df)
hpwren_station_null_per_df

# via trial and error there are 4 hpwren stations with > 20% null percentage for wind speed; these will be manually filtered out and reprocessed
# HP003, HP006 (100%), HP015, HP025

CPU times: user 2.72 s, sys: 83.5 ms, total: 2.81 s
Wall time: 2.81 s


,Date_Time,air_temp_set_1,relative_humidity_set_1,wind_speed_set_1,wind_gust_set_1,wind_direction_set_1,dew_point_temperature_set_1d,u,v
,Unnamed: 1_level_1,Celsius,%,m/s,m/s,Degrees,Celsius,Unnamed: 8_level_1,Unnamed: 9_level_1
Station_ID,,,,,,,,,
HP001,0,1.303828,1.303828,7.465208,7.465208,7.470773,1.303828,7.470773,7.470773
HP002,0,0.639913,0.640353,6.643720,6.644161,6.645483,0.640353,6.645483,6.645483
HP004,0,1.308613,1.308210,0.290221,0.290221,0.305517,1.309015,0.305517,0.305517
HP005,0,0.857593,0.857593,0.013425,0.013425,0.020532,0.857593,0.020532,0.020532
HP007,0,1.192222,1.192222,0.005503,0.005896,0.022013,1.192222,0.022013,0.022013
HP009,0,0.912596,0.912596,0.425801,0.426377,0.513263,0.912596,0.513839,0.513839
HP010,0,0.779617,0.779617,0.066570,0.067039,0.080165,0.779617,0.080165,0.080165
HP014,0,1.048326,1.050880,3.025376,3.028355,3.025802,1.051305,3.025802,3.025802


#### SC-EDISON

In [13]:
weather_scedison_df.shape

(6720705, 10)

In [14]:
%%time
calc_att_null_miss_percentage(weather_scedison_df).round(2)

CPU times: user 1.61 s, sys: 22.9 ms, total: 1.63 s
Wall time: 1.63 s


,,null_miss_percentage
Station_ID,Unnamed: 0_level_1,62.65
Date_Time,Unnamed: 1_level_1,62.65
air_temp_set_1,Celsius,62.69
relative_humidity_set_1,%,62.68
wind_speed_set_1,m/s,62.68
wind_gust_set_1,m/s,62.67
wind_direction_set_1,Degrees,63.40
dew_point_temperature_set_1d,Celsius,62.69
u,Unnamed: 8_level_1,63.43
v,Unnamed: 9_level_1,63.43


In [15]:
%%time
scedison_station_null_per_df = calc_station_null_percentage(weather_scedison_df)
scedison_station_null_per_df

CPU times: user 7.58 s, sys: 484 ms, total: 8.06 s
Wall time: 8.07 s


,Date_Time,air_temp_set_1,relative_humidity_set_1,wind_speed_set_1,wind_gust_set_1,wind_direction_set_1,dew_point_temperature_set_1d,u,v
,Unnamed: 1_level_1,Celsius,%,m/s,m/s,Degrees,Celsius,Unnamed: 8_level_1,Unnamed: 9_level_1
Station_ID,,,,,,,,,
SE003,0,0.000000,0.004751,0.061763,0.211418,0.043551,0.004751,0.105313,0.105313
SE004,0,0.000000,0.000621,0.080707,0.000000,0.560605,0.000621,0.641312,0.641312
SE007,0,0.000000,0.000000,0.087918,0.000000,1.558379,0.000000,1.646297,1.646297
SE009,0,1.702043,0.021647,0.088442,0.000000,0.408812,1.723690,0.497254,0.497254
SE014,0,0.000000,0.000618,0.051916,0.000000,8.273228,0.000618,8.325144,8.325144
SE018,0,0.000000,0.000619,0.076771,0.000000,0.265602,0.000619,0.342372,0.342372
SE024,0,0.000000,0.000681,0.071552,0.000000,0.089269,0.000681,0.160821,0.160821
SE030,0,0.000000,0.000000,0.059003,0.001255,8.314398,0.000000,8.373401,8.373401


## Check wind direction nulls

Is wind direction null only when wind speed is 0? -> Primarily yes. Wind speed=0 accounts for 99.99% of null wind directions.

In [16]:
weather_sdge_df[weather_sdge_df["wind_direction_set_1"]["Degrees"].isna()][
    "wind_speed_set_1"
]["m/s"].value_counts(normalize=True) * 100

0.000    99.988184
1.327     0.000215
1.235     0.000215
1.199     0.000215
1.980     0.000215
0.468     0.000215
1.214     0.000107
0.350     0.000107
0.653     0.000107
0.952     0.000107
2.450     0.000107
2.710     0.000107
1.550     0.000107
3.730     0.000107
2.546     0.000107
4.337     0.000107
2.090     0.000107
2.550     0.000107
0.319     0.000107
0.360     0.000107
0.741     0.000107
0.273     0.000107
1.451     0.000107
0.980     0.000107
1.460     0.000107
3.420     0.000107
0.607     0.000107
1.394     0.000107
1.020     0.000107
0.535     0.000107
1.163     0.000107
3.395     0.000107
4.522     0.000107
1.615     0.000107
2.176     0.000107
1.374     0.000107
3.010     0.000107
0.808     0.000107
1.150     0.000107
1.200     0.000107
0.818     0.000107
4.970     0.000107
0.268     0.000107
1.152     0.000107
0.427     0.000107
7.259     0.000107
0.823     0.000107
4.491     0.000107
2.269     0.000107
3.800     0.000107
0.561     0.000107
1.970     0.000107
1.000     0.

In [17]:
weather_hpwren_df[weather_hpwren_df["wind_direction_set_1"]["Degrees"].isna()][
    "wind_speed_set_1"
]["m/s"].value_counts(normalize=True) * 100

0.0    100.0
Name: m/s, dtype: float64

In [18]:
weather_scedison_df[weather_scedison_df["wind_direction_set_1"]["Degrees"].isna()][
    "wind_speed_set_1"
]["m/s"].value_counts(normalize=True) * 100

0.000    99.873070
1.420     0.002214
0.130     0.001476
0.777     0.001476
0.828     0.001476
3.870     0.001476
0.499     0.001476
0.314     0.001476
0.093     0.001476
0.880     0.001476
2.289     0.001476
1.821     0.001476
0.110     0.001476
0.210     0.001476
0.310     0.001476
1.390     0.001476
0.010     0.001476
3.498     0.000738
4.060     0.000738
4.030     0.000738
0.530     0.000738
1.120     0.000738
1.660     0.000738
2.464     0.000738
1.980     0.000738
1.456     0.000738
1.670     0.000738
0.463     0.000738
0.365     0.000738
0.120     0.000738
0.911     0.000738
3.530     0.000738
0.736     0.000738
2.629     0.000738
3.790     0.000738
3.951     0.000738
3.990     0.000738
0.715     0.000738
3.370     0.000738
0.820     0.000738
1.680     0.000738
2.070     0.000738
3.025     0.000738
0.077     0.000738
1.528     0.000738
2.650     0.000738
1.360     0.000738
0.792     0.000738
2.634     0.000738
2.624     0.000738
0.890     0.000738
1.209     0.000738
0.936     0.

### Read in processed data

In [19]:
%%time
weather_sdge_df2 = pd.read_csv("../../data/processed/weather_SDGE.csv", header=[0, 1])
weather_hpwren_df2 = pd.read_csv(
    "../../data/processed/weather_HPWREN.csv", header=[0, 1]
)
weather_scedison_df2 = pd.read_csv(
    "../../data/processed/weather_SC-EDISON.csv", header=[0, 1]
)

CPU times: user 30.8 s, sys: 3.16 s, total: 34 s
Wall time: 34 s


### Concat to single weather dataframe

In [20]:
weather_df2 = pd.concat([weather_sdge_df2, weather_hpwren_df2, weather_scedison_df2])

### Convert col to datetime

In [21]:
weather_df2["Date_Time"] = pd.to_datetime(
    weather_df2["Date_Time"]["Unnamed: 1_level_1"]
)

weather_df2["Date_Time"].head()

,Unnamed: 1_level_1
0,2016-06-03 00:00:00+00:00
1,2016-06-03 00:10:00+00:00
2,2016-06-03 00:20:00+00:00
3,2016-06-03 00:30:00+00:00
4,2016-06-03 00:40:00+00:00


### Check interval counts

Are all stations on 10 minute intervals after processing?

In [22]:
weather_df2["Date_Time"]["Unnamed: 1_level_1"].dt.minute.value_counts()

20    5592152
40    5591816
30    5591640
50    5591492
0     5590839
10    5540861
Name: Unnamed: 1_level_1, dtype: int64